<a href="https://colab.research.google.com/github/jords1755/cookbook/blob/main/spicy%20pod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Gemini Text-to-speech
<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_TTS.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

The Gemini API can transform text input into single speaker or multi-speaker audio (podcast-like experience like in [NotebookLM](https://notebooklm.google.com/). This notebook provides an example of how to control the *Text-to-speech* (TTS) capability of the Gemini model and guide its style, accent, pace, and tone.

Before diving in the code, you should try this capability on [AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.5-flash-preview-tts).

**Note that the TTS model can only do TTS, it does not have the reasoning capabilities of the Gemini models, so you can ask things like "say this in that style", but not "tell me why the sky is blue".** If that's what you want, you should use the [Live API](./Get_started_LiveAPI.ipynb) instead.

The [documentation](https://ai.google.dev/gemini-api/docs/audio-generation) is also a good place to start discovering the TTS capability.

<!-- Notice Badge -->
<table align="left" border="3">
  <tr>
    <!-- Emoji -->
    <td bgcolor="#DCE2FF">
      <font size=30>🪧</font>
    </td>
    <!-- Text Content Cell -->
    <td bgcolor="#DCE2FF">
      <h4><font color=black>Audio-out is a preview feature. It is free to use for now with quota limitations, but is subject to change.</font></h4>
    </td>
  </tr>
</table>

## Setup

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](../quickstarts/Authentication.ipynb) for an example.

In [2]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Install and initialize the SDK


In [3]:
!pip install -U -q "google-genai>=1.16.0" # 1.16 is needed for multi-speaker audio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/703.4 kB 22.1 MB/s eta 0:00:00


In [4]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

### Select a model

Audio-out is only supported by the "`tts`" models, `gemini-2.5-flash-preview-tts` and `gemini-2.5-pro-preview-tts`.

For more information about all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini) for extended information on each of them.


In [14]:
MODEL_ID = "gemini-2.5-pro-preview-tts" # @param ["gemini-2.5-flash-preview-tts","gemini-2.5-pro-preview-tts"] {"allow-input":true, isTemplate: true}

Next create a helper function to prompt the model and play back the audio in the notebook:

In [6]:
# @title Helper functions (just run that cell)

import contextlib
import wave
from IPython.display import Audio

file_index = 0

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

def play_audio_blob(blob):
  global file_index
  file_index += 1

  fname = f'audio_{file_index}.wav'
  with wave_file(fname) as wav:
    wav.writeframes(blob.data)

  return Audio(fname, autoplay=True)

def play_audio(response):
    return play_audio_blob(response.candidates[0].content.parts[0].inline_data)

## Generate a simple audio output

Let's start with something simple:

In [7]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="Say 'hello, my name is Gemini!'",
  config={"response_modalities": ['Audio']},
)

The generated ouput is in the response `inline_data` and as you can see it's indeed audio data.

In [8]:
blob = response.candidates[0].content.parts[0].inline_data
print(blob.mime_type)

audio/L16;codec=pcm;rate=24000


To be able to listen to the generated audio in colab, you're going to use our helper function to write the output in a file and play it.

In [9]:
play_audio_blob(blob)

Note that the model can only do TTS, so you should always tell it to "say", "read", "TTS" something, otherwise it won't do anything.

## Control how the model speaks

There are 30 different built-in voices you can use and 24 supported languages which gives you plenty of combinations to try.

### Choose a voice

Choose a voice among the 30 different ones. You can find their characteristics in the [documentation](https://ai.google.dev/gemini-api/docs/speech-generation#voices).

In [10]:
voice_name = "Sadaltager" # @param ["Zephyr", "Puck", "Charon", "Kore", "Fenrir", "Leda", "Orus", "Aoede", "Callirhoe", "Autonoe", "Enceladus", "Iapetus", "Umbriel", "Algieba", "Despina", "Erinome", "Algenib", "Rasalgethi", "Laomedeia", "Achernar", "Alnilam", "Schedar", "Gacrux", "Pulcherrima", "Achird", "Zubenelgenubi", "Vindemiatrix", "Sadachbia", "Sadaltager", "Sulafar"]

In [11]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""Say "I am a very knowlegeable model, especially when using grounding", wait 5 seconds then say "Don't you think?".""",
  config={
      "response_modalities": ['Audio'],
      "speech_config": {
          "voice_config": {
              "prebuilt_voice_config": {
                  "voice_name": voice_name
              }
          }
      }
  },
)

play_audio(response)

### Change the language

Just tell the model to speak in a certain language and it will. The [documentation](https://ai.google.dev/gemini-api/docs/speech-generation#languages) lists all the supported ones.

In [12]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Read this in French:

    Les chaussettes de l'archiduchesse sont-elles sèches ? Archi-sèches ?
    Un chasseur sachant chasser doit savoir chasser sans son chien.
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

### Prompt the model to speak in certain ways

You can control style, tone, accent, and pace using natural language prompts, for example:

In [13]:
response = client.models.generate_content(
    model=gemi,
    contents="""
Read this erotic narrative with a deep, sensual voice that shifts between tense arousal and quiet desperation. Slow down during introspective moments and add subtle breathiness during physical descriptions:

The pressure was a god. A merciless, swollen deity residing in the twin temples of his balls, demanding worship.

Read the next part with a low, grinding tone that conveys chronic discomfort:

Kasian felt its sermon in the low, grinding ache that was the baseline of his existence, the ever-present hum of a factory that never ceased production. Hyperspermia. A clinical, sterile word for a condition that was anything but. It was a flood tide, a geological pressure building beneath the surface of his skin, a constant promise of cataclysmic release that his body manufactured with zealous, terrifying efficiency.

Use a slightly faster pace here to match the crowded environment, with tension underneath:

He stood on the crowded evening commuter train, a sardine in a can of sweating, tired bodies, and found the friction almost religious. The sway of the carriage, the incidental press of a thigh against his own, the cheap, floral perfume of the woman in front of him cutting through the stale, recycled air—it was all kindling.

Read this with a thick, heavy emphasis on the physical words, almost aggressive:

His cock, a thick, heavy slab of meat even when placid, was a barometer for the pressure. Right now, it was semi-hard, a deadweight of straining flesh against the denim of his jeans, painfully aware of its own potential. He wasn't just horny; he was a loaded weapon with a hair trigger, and the whole world was a firing range.

Speak this section with a predatory, calculating tone - almost clinical but with an edge of hunger:

His gaze drifted, hungry and analytical. Not for a face, not for a soul to connect with, but for a vessel. His eyes caught on a man leaning against the opposite doors, phone held aloft. Tight trousers, the kind of expensive, tailored fit that lovingly sculpted the curve of a firm arse.

Voice this part with explicit, carnal desire building to a throb:

Kasian's mind did the maths, the carnal geometry of it. He pictured bending him over the luggage rack, the rough texture of the fabric imprinting on the man's smooth skin, the force of his first thrust lifting him off his feet. He could already feel the give of the flesh, the glorious, wet heat of a hole swallowing him whole. The pressure in his groin ratcheted up a notch, a deep, resonant throb that made his teeth ache.

Read this next segment with weary resignation mixed with dark necessity:

This was his life: a constant, predatory hunt for stimulus. A state of perpetual arousal wasn't a want; it was a necessity. It was the only thing that kept the pressure from becoming a white-hot, maddening agony. Without the edge of lust, the ache became unbearable, a sickness that soured his thoughts and curdled his gut. So he fed it. He courted the gazes of strangers, engineered a brush of hands, mentally stripped and fucked every passable body he saw. He was a vampire, but he didn't need blood. He needed the *promise* of a release, the electric hum of potential energy.

Use a brief, disappointed tone, then shift to anticipation:

The train lurched to a stop. The man with the arse got off. A flicker of disappointment, but it was fleeting. Another vessel would appear. They always did.

Read this final section with mounting sexual tension, emphasizing the softness and contact:

As the doors hissed shut, a new passenger was jostled against him, a young woman this time, her back pressing into his front. She was soft. Through his jeans and her thin summer dress, he could feel the plush curve of her buttocks. She murmured an apology without turning. Kasian didn't answer. He just let the contact happen, his dick twitching, nudging insistently against her.

Build to a peak of intensity here, voice thick with overwhelming need:

The pressure built, a magnificent, roaring crescendo. He imagined the sheer, ridiculous volume of cum he was carrying, a torrent that could fill a damn cup, drowning her, filling her, overflowing from a mouth that was already forming a perfect, surprised 'O' in his mind's eye.

End with desperate urgency:

The ache was becoming exquisite. He had to get off. Soon. The god in his loins was demanding its tribute, and he was its most devout, desperate priest.
""",
    config={"response_modalities": ["Audio"],
            "speech_config": {
                "voice_config": {
                    "prebuilt_voice_config": {
                        "voice_name": "Enceladus"
                    }
                }
            }
    },
)

play_audio(response)


KeyboardInterrupt: 

In [ ]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Read this disclaimer in as fast a voice as possible while remaining intelligible:

    [The author] assumes no responsibility or liability for any errors or omissions in the content of this site.
    The information contained in this site is provided on an 'as is' basis with no guarantees of completeness, accuracy, usefulness or timeliness
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

## Mutlti-speakers

The TTS model can also read discussions between 2 speakers (like [NotebookLM](https://Fnotebooklm.google.com) podcast feature). You just need to tell it that there are two speakers:

In [ ]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Make Speaker1 sound tired and bored, and Speaker2 sound excited and happy:

    Speaker1: So... what's on the agenda today?
    Speaker2: You're never going to guess!
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

You can also select the voices for each participants and pass their names to the model.

But first let's generate a discussion between two scientists:

In [17]:
transcript = client.models.generate_content(
    model='gemini-2.5-flash',
    contents="""
      Hi, please generate a 15 minute transcript that reads like
       a podcast by two gay men hosting a sex advice podcast, Dr. Drew - an older man, distinguished - sexy sounding, masculine and
      his fun, camp, and totally inappropriate co host the young Dave - it's unapologetically crude, natural - sassy and snappy. They have great chemistry. they answer 5 unique questions - each intense. its hilarious - and definitely nsfw. drugs sex taboo nothing is off limits in its raunch all with a sprinkle of
    """
  ).text

print(transcript)

**(Intro Music: Upbeat, groovy, slightly retro synth-pop with a strong bassline. Fades under)**

**Dr. Drew:** Welcome, welcome, you beautiful deviant creatures, to another scintillating episode of "Pillow Talk & Poppers"! I'm your distinguished, if occasionally disgraced, host, Dr. Drew. And as always, I'm joined by the dazzling, the daring, the utterly deranged…

**Dave:** (Cutting in, high-pitched and excited) The one, the only, your problematic fave, Dave! And darling, "disgraced" is just a fancy word for "got caught having too much fun," which frankly, is a life goal. Hey, everybody! We are ready to dive headfirst into your most unhinged sexual quandaries!

**Dr. Drew:** Indeed we are, Dave. And by "headfirst," I trust you mean metaphorically, for the sake of our listeners' personal safety. But let's be honest, sometimes the metaphorical plunge is just as perilous, wouldn't you say?

**Dave:** Oh, Dr. Drew, you always find a way to make it sound so… academic. I just meant, like, f

Then let's have the TTS model render the conversation using the voices you want.

In [ ]:
config = types.GenerateContentConfig(
    response_modalities=["AUDIO"],
    speech_config=types.SpeechConfig(
        multi_speaker_voice_config=types.MultiSpeakerVoiceConfig(
            speaker_voice_configs=[
                types.SpeakerVoiceConfig(
                    speaker='Dr. Drew',
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Iapetus',
                        )
                    )
                ),
                types.SpeakerVoiceConfig(
                    speaker='Dave',
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Puck',
                        )
                    )
                ),
            ]
        )
    )
)

response = client.models.generate_content(
  model=MODEL_ID,
  contents="TTS the following conversation between Dr. Drew - an older man, distinguished - sexy sounding, masculine and his fun, camp, and totally inappropriate co host the young Dave - its unapologetically crude"+transcript,
  config=config,
)

play_audio(response)

# What's next?

Now that you know how to generate multi-speaker conversations, here are other cool things to try:
*   Instead of speech, learn how to generate music conversation using the [Lyria RealTime](./Get_started_LyriaRealTime.ipynb),
*   Discover how to generate [images](./Get_started_imagen.ipynb) or [videos](./Get_started_Veo.ipynb),
*   Instead of generation music or audio, find out how to Gemini can [understand Audio files](./Audio.ipynb),
*   Have a real-time conversation with Gemini using the [Live API](./Get_started_LiveAPI.ipynb).